In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate, optimize
import ddm
import os 
import csv

In [64]:
class LossWLS(ddm.LossFunction):
    name = 'Weighted least squares as described in Ratcliff & Tuerlinckx 2002'
    rt_quantiles = [0.1, 0.3, 0.5, 0.7, 0.9]
    rt_q_weights = [2, 2, 1, 1, 0.5]
#     exp_data = pd.read_csv('../data/measures.csv', 
#                             usecols=['subj_id', 'RT', 'is_turn_decision', 'tta_condition', 'd_condition'])
           
    def setup(self, dt, T_dur, **kwargs):
        self.dt = dt
        self.T_dur = T_dur
    
    def get_rt_quantiles(self, x, t_domain, exp=False):
        cdf = x.cdf_corr(T_dur=self.T_dur, dt=self.dt) if exp else x.cdf_corr()
        cdf_interp = interpolate.interp1d(t_domain, cdf/cdf[-1])
        # If the model produces very fast RTs, interpolated cdf(0) can be >0.1, then we cannot find root like usual
        # In this case, the corresponding rt quantile is half of the time step of cdf
        rt_quantile_values = [optimize.root_scalar(lambda x:cdf_interp(x)-quantile, bracket=(0, t_domain[-1])).root
                              if (cdf_interp(0)<quantile) else self.dt/2
                              for quantile in self.rt_quantiles]
        return np.array(rt_quantile_values)
    
    def loss(self, model):
        solultions = self.cache_by_conditions(model)
        WLS = 0
        for comb in self.sample.condition_combinations(required_conditions=self.required_conditions):
            c = frozenset(comb.items())
            comb_sample = self.sample.subset(**comb)
            WLS += 4*(solultions[c].prob_correct() - comb_sample.prob_correct())**2            
            # Sometimes model p_correct is very close to 0, then RT distribution is weird, in this case ignore RT error 
            if ((solultions[c].prob_correct()>0.001) & (comb_sample.prob_correct()>0)):
#                 self.condition_data = self.exp_data[(self.exp_data.d_condition==comb['d_condition']) 
#                                                     & (self.exp_data.tta_condition==comb['tta_condition'])]
#                 self.condition_data = list(comb_sample.items(correct=True))
                model_rt_q = self.get_rt_quantiles(solultions[c], model.t_domain(), exp=False)
                exp_rt_q = self.get_rt_quantiles(comb_sample, model.t_domain(), exp=True)
                WLS += np.dot((model_rt_q-exp_rt_q)**2, self.rt_q_weights)*comb_sample.prob_correct()
        return WLS

In [65]:
T_dur = 2.5

class DriftTtaDistance(ddm.models.Drift):
    name = 'Drift depends on TTA and distance'
    required_parameters = ['alpha', 'tta_crit', 'beta', 'd_crit']
    required_conditions = ['tta_condition', 'd_condition'] 
    
    def get_drift(self, t, conditions, **kwargs):
        v = conditions['d_condition']/conditions['tta_condition']
        return (self.alpha*(conditions['tta_condition'] - t - self.tta_crit + 
                            self.beta*(conditions['d_condition'] - v*t - self.d_crit)))

class BoundCollapsingTta(ddm.models.Bound):
    name = 'Bounds collapsing with TTA'
    required_parameters = ['b_0', 'k', 'tta_crit']
    required_conditions = ['tta_condition'] 
    def get_bound(self, t, conditions, **kwargs):
        tau = conditions['tta_condition'] - t
        return self.b_0/(1+np.exp(-self.k*(tau-self.tta_crit)))

param_names = ['alpha', 'tta_crit', 'beta', 'd_crit', 'noise', 'b_0', 'k', 'tta_crit', 'nondectime', 'halfwidth']
tta_crit = ddm.Fittable(minval=3, maxval=6)
model_TTA_bounds = ddm.Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time',
                             drift=DriftTtaDistance(alpha=ddm.Fittable(minval=0.1, maxval=3),
                                                     tta_crit=tta_crit,
                                                     beta=ddm.Fittable(minval=0, maxval=1),
                                                     d_crit=ddm.Fittable(minval=90, maxval=150)),
                             noise=ddm.NoiseConstant(noise=1),
                             bound=BoundCollapsingTta(b_0=ddm.Fittable(minval=0.5, maxval=5), 
                                                      k=ddm.Fittable(minval=0.1, maxval=2),
                                                      tta_crit=tta_crit),
                             overlay=ddm.OverlayNonDecisionUniform(nondectime=ddm.Fittable(minval=0, maxval=0.5),
                                                                   halfwidth=ddm.Fittable(minval=0, maxval=0.3)),
                             T_dur=T_dur)

In [95]:
def fit_model(model, exp_data, subj_id, condition):
    print(subj_id)
    print(condition)
    training_data = exp_data[exp_data.subj_id == subj_id 
                             & ~((exp_data.d_condition==condition['d']) & (exp_data.tta_condition==condition['tta']))]
    training_sample = ddm.Sample.from_pandas_dataframe(df=training_data, 
                                                       rt_column_name='RT', correct_column_name='is_turn_decision')
    return(ddm.fit_adjust_model(sample=exp_sample, model=model, lossfunction=LossWLS))

def write_to_csv(directory, filename, array):
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(os.path.join(directory, filename), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(array)

In [ ]:
model = model_TTA_bounds

exp_data = pd.read_csv('../data/measures.csv', usecols=['subj_id', 'RT', 'is_turn_decision', 
                                                        'tta_condition', 'd_condition'])
subjects = exp_data.subj_id.unique()
conditions = [{'tta': tta, 'd': d} 
                  for tta in exp_data.tta_condition.unique() 
                  for d in exp_data.d_condition.unique()]

directory = '../model_fit_results/cross-validation'
write_to_csv(directory, 'model_%s_params.csv' % (model.name[0]), 
             ['subj_id', 'tta', 'd', 'loss'] + param_names)

for subj_id in subjects:
    for condition in conditions:
        fitted_model = fit_model(model, exp_data, subj_id, condition)
        write_to_csv(directory, 'model_%s_params.csv' % (model.name[0]), 
                     [subj_id, condition['tta'], condition['d'], fit_model.get_fit_result().value()] 
                     + fit_model.get_model_parameters())

129
{'tta': 4.0, 'd': 90.0}
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.0352343765663041, minval=0.1, maxval=3), tta_crit=Fitted(4.330878057468891, minval=3, maxval=6), beta=Fitted(0.37796675526355455, minval=0, maxval=1), d_crit=Fitted(96.61959494107825, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(2.7501464636905366, minval=0.5, maxval=5), k=Fitted(1.670213393241525, minval=0.1, maxval=2), tta_crit=Fitted(4.330878057468891, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.4065962520290739, minval=0, maxval=0.5), halfwidth=Fitted(0.23749703842228836, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=6.66982469935958
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.231411571527723, minval=0.1, maxval=3), 

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.0782130294958663, minval=0.1, maxval=3), tta_crit=Fitted(5.2733487391852405, minval=3, maxval=6), beta=Fitted(0.7879113503044012, minval=0, maxval=1), d_crit=Fitted(120.00292162089674, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(1.6878481507684637, minval=0.5, maxval=5), k=Fitted(0.5957356861169533, minval=0.1, maxval=2), tta_crit=Fitted(5.2733487391852405, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.25614259906904696, minval=0, maxval=0.5), halfwidth=Fitted(0.24868096225061948, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=5.493579691440781
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.472854296632082, minval=0.1, maxval=3), tta_crit=Fitted(3.31983

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.4345842991970785, minval=0.1, maxval=3), tta_crit=Fitted(4.0135032210629324, minval=3, maxval=6), beta=Fitted(0.8715718500249884, minval=0, maxval=1), d_crit=Fitted(144.60308651236957, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(1.2151248567023842, minval=0.5, maxval=5), k=Fitted(1.8212366083223164, minval=0.1, maxval=2), tta_crit=Fitted(4.0135032210629324, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.3534169981152065, minval=0, maxval=0.5), halfwidth=Fitted(0.11522537421565626, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=3.2930871524411915
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.3013275331418843, minval=0.1, maxval=3), tta_crit=Fitted(5.3374

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.4855296349029548, minval=0.1, maxval=3), tta_crit=Fitted(3.2221809197849067, minval=3, maxval=6), beta=Fitted(0.03995766022758224, minval=0, maxval=1), d_crit=Fitted(138.90976712534453, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(3.82100575897374, minval=0.5, maxval=5), k=Fitted(1.4421740211991596, minval=0.1, maxval=2), tta_crit=Fitted(3.2221809197849067, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.1720529710898158, minval=0, maxval=0.5), halfwidth=Fitted(0.1861561849397286, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=11.819306058793991
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.7697374536428812, minval=0.1, maxval=3), tta_crit=Fitted(3.761676

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(2.9432547694625453, minval=0.1, maxval=3), tta_crit=Fitted(3.84522967756063, minval=3, maxval=6), beta=Fitted(0.6773766522029988, minval=0, maxval=1), d_crit=Fitted(148.89521252648092, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(3.891763497188938, minval=0.5, maxval=5), k=Fitted(0.9768790994402676, minval=0.1, maxval=2), tta_crit=Fitted(3.84522967756063, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.09107774855800391, minval=0, maxval=0.5), halfwidth=Fitted(0.2879185719057585, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=9.32746483722145
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.3991558831584079, minval=0.1, maxval=3), tta_crit=Fitted(4.67837597807

...

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.1085431757918549, minval=0.1, maxval=3), tta_crit=Fitted(3.043325892799924, minval=3, maxval=6), beta=Fitted(0.21667729392451984, minval=0, maxval=1), d_crit=Fitted(141.67961825643627, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(4.28516390085686, minval=0.5, maxval=5), k=Fitted(1.9225300376527983, minval=0.1, maxval=2), tta_crit=Fitted(3.043325892799924, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.3195704605591979, minval=0, maxval=0.5), halfwidth=Fitted(0.12818137354500297, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=45.897257056404996
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.1901510403264262, minval=0.1, maxval=3), tta_crit=Fitted(3.2522158

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.1298829544105558, minval=0.1, maxval=3), tta_crit=Fitted(3.419943572203377, minval=3, maxval=6), beta=Fitted(0.8397771032549243, minval=0, maxval=1), d_crit=Fitted(133.30429760830214, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(3.0606224959229396, minval=0.5, maxval=5), k=Fitted(0.24366513151835356, minval=0.1, maxval=2), tta_crit=Fitted(3.419943572203377, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.09720040931834367, minval=0, maxval=0.5), halfwidth=Fitted(0.2859138251704645, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=5.108531812303758
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.3123069466742383, minval=0.1, maxval=3), tta_crit=Fitted(3.889093

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.1293956873219226, minval=0.1, maxval=3), tta_crit=Fitted(3.070099672389146, minval=3, maxval=6), beta=Fitted(0.08327597682250043, minval=0, maxval=1), d_crit=Fitted(114.81595649617076, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(2.1670613327573465, minval=0.5, maxval=5), k=Fitted(1.626056152136509, minval=0.1, maxval=2), tta_crit=Fitted(3.070099672389146, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.10614164316796956, minval=0, maxval=0.5), halfwidth=Fitted(0.26888629398896724, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=2.3242919208094674
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.0448181542848098, minval=0.1, maxval=3), tta_crit=Fitted(3.91059

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(2.560834856967606, minval=0.1, maxval=3), tta_crit=Fitted(3.303585470474694, minval=3, maxval=6), beta=Fitted(0.04185976633526767, minval=0, maxval=1), d_crit=Fitted(135.41966870172124, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(2.052342822834163, minval=0.5, maxval=5), k=Fitted(0.19190861012439187, minval=0.1, maxval=2), tta_crit=Fitted(3.303585470474694, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.05091998910914214, minval=0, maxval=0.5), halfwidth=Fitted(0.21317635102179755, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=3.060013904562153
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.4338043097617892, minval=0.1, maxval=3), tta_crit=Fitted(4.958983

Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(1.0403678540591432, minval=0.1, maxval=3), tta_crit=Fitted(5.3123799594726835, minval=3, maxval=6), beta=Fitted(0.4003600848269636, minval=0, maxval=1), d_crit=Fitted(147.35970171661617, minval=90, maxval=150)), noise=NoiseConstant(noise=1), bound=BoundCollapsingTta(b_0=Fitted(1.2519546534400574, minval=0.5, maxval=5), k=Fitted(0.3283712071965871, minval=0.1, maxval=2), tta_crit=Fitted(5.3123799594726835, minval=3, maxval=6)), IC=ICPointSourceCenter(), overlay=OverlayNonDecisionUniform(nondectime=Fitted(0.1867117794559951, minval=0, maxval=0.5), halfwidth=Fitted(0.2014277923955976, minval=0, maxval=0.3)), dx=0.005, dt=0.005, T_dur=2.5) loss=6.746677271404695
Model(name='5 TTA- and d-dependent drift and bounds and uniformly distributed nondecision time', drift=DriftTtaDistance(alpha=Fitted(0.2791799642912951, minval=0.1, maxval=3), tta_crit=Fitted(5.561329